## 🔢 Description Embedding

In [1]:
# Imports
from   dotenv   import load_dotenv
import pandas   as pd
import datetime
import sys
import os

# Add the upper folder to sys.path
import LLMUtils
import Preprocessing

#### Initialization

In [2]:
print("⚡ Start - {} ⚡\n".format(datetime.datetime.now()))
startTime = datetime.datetime.now()

⚡ Start - 2024-07-20 15:11:41.193118 ⚡



#### 📥 1) Load Data 

In [3]:
DATA_PATH = "../../0_Data/0_AndroCatSet.csv"
#DATA_PATH = "../../0_Data/1_AndroCatSetMini.csv"

# Read the data
appsDF = pd.read_csv(DATA_PATH)
appsDF = appsDF.drop(columns=['googlePlayCategoryID'])

# TEST
#appsDF = appsDF.head(2)

print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

--- #️⃣ Apps: 5000 


In [4]:
appsDF.head(5)

,sha256,pkgName,classID,googlePlayDescription
0,9B30837BD2474AC3623A43D052F7ADC4C63E4AA9981F0F...,my.android.calc,Calculator,Handiness universal percentage calculator for ...
1,686DE8D8A0D08992CB135BC7A0500D0109D9697A1140B8...,com.vpn.basiccalculator,Calculator,CITIZEN CALCULATOR by ANGEL NX is best Mobile ...
2,A49864DCC90F6730569455BDFA39B4B7CF70AE0C34D656...,com.ba.fractioncalculator,Calculator,"<b>Free offline fraction calculator</b> ✌, sup..."
3,85C80B7ED3799C04CBD107DD3004F91ED4DC2BF8D75C35...,de.sfr.calctape,Calculator,What happens if you combine a standard calcula...
4,66442DEF269FF22FB4177759343D973DA1A6A3AFE2ECE3...,com.digitalchemy.calculator.freefraction,Calculator,Fraction Calculator Plus is your best way to d...


#### 🔢 2) Preprocess Description

In [5]:
appsDF['description'] = appsDF['googlePlayDescription'].apply(Preprocessing.preprocessDescription)
appsDF = appsDF.drop(columns=['googlePlayDescription'])
appsDF.head(5)

,sha256,pkgName,classID,description
0,9B30837BD2474AC3623A43D052F7ADC4C63E4AA9981F0F...,my.android.calc,Calculator,handiness universal percentage calculator simp...
1,686DE8D8A0D08992CB135BC7A0500D0109D9697A1140B8...,com.vpn.basiccalculator,Calculator,citizen calculator angel best mobile app world...
2,A49864DCC90F6730569455BDFA39B4B7CF70AE0C34D656...,com.ba.fractioncalculator,Calculator,free offline fraction calculator support also ...
3,85C80B7ED3799C04CBD107DD3004F91ED4DC2BF8D75C35...,de.sfr.calctape,Calculator,happens combine standard calculator app classi...
4,66442DEF269FF22FB4177759343D973DA1A6A3AFE2ECE3...,com.digitalchemy.calculator.freefraction,Calculator,fraction calculator plus best way deal everyda...


#### 🔢 3) Generate Numerical Embeddings

In [6]:
# To interact with ChatGPT
gptManager = LLMUtils.GptManager()

$ Pay here

In [7]:
# Generate Embeddings
appsDF['embedding'] = appsDF['description'].apply(gptManager.generateEmbedding)
appsDF.head(5)

,sha256,pkgName,classID,description,embedding
0,9B30837BD2474AC3623A43D052F7ADC4C63E4AA9981F0F...,my.android.calc,Calculator,handiness universal percentage calculator simp...,"[0.00032980614923872054, -0.00386972539126873,..."
1,686DE8D8A0D08992CB135BC7A0500D0109D9697A1140B8...,com.vpn.basiccalculator,Calculator,citizen calculator angel best mobile app world...,"[-0.0048510185442864895, -0.005108269397169352..."
2,A49864DCC90F6730569455BDFA39B4B7CF70AE0C34D656...,com.ba.fractioncalculator,Calculator,free offline fraction calculator support also ...,"[-0.03689534589648247, 0.0034220372326672077, ..."
3,85C80B7ED3799C04CBD107DD3004F91ED4DC2BF8D75C35...,de.sfr.calctape,Calculator,happens combine standard calculator app classi...,"[0.01341024786233902, -0.0020422476809471846, ..."
4,66442DEF269FF22FB4177759343D973DA1A6A3AFE2ECE3...,com.digitalchemy.calculator.freefraction,Calculator,fraction calculator plus best way deal everyda...,"[-0.014099321328103542, 0.0014035949716344476,..."


### 🔢 4) Save into CSV

In [8]:
appsDF.to_csv("../TmpData/0_AndroCatSetEmbeddings.csv", index=False)

##### 🔚 End

In [9]:
endTime = datetime.datetime.now()
print("\n🔚 --- End - {} --- 🔚".format(endTime))

# Assuming endTime and startTime are in seconds
totalTime = endTime - startTime
minutes = totalTime.total_seconds() // 60
seconds = totalTime.total_seconds() % 60
print("⏱️ --- Time: {:02d} minutes and {:02d} seconds --- ⏱️".format(int(minutes), int(seconds)))


🔚 --- End - 2024-07-20 15:38:03.598827 --- 🔚
⏱️ --- Time: 26 minutes and 22 seconds --- ⏱️
